<a href="https://colab.research.google.com/github/varunsalunkhe/News-Research-Tool-using-LLM/blob/main/News_Research_Tool_using_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# !pip install ctransformers
# !pip install chromadb
# !pip install langchain
# !pip install sentence_transformers
# !pip install bitsandbytes
# !pip install unstructured
# !pip install transformers
# !pip install langchain_community

In [4]:
#loading URL's
import pandas
from langchain_community.document_loaders import UnstructuredURLLoader

urls = [
    "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-8-2023",
    "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-9-2023",
]

print(urls)

['https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-8-2023', 'https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-9-2023']


In [5]:
#Loading the data from URLs

loader = UnstructuredURLLoader(urls=urls)

print(loader)

data = loader.load()

# print(data)

In [6]:
# LangChain
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.llms import CTransformers

from transformers import AutoModelForCausalLM

# Hiding warnings
import warnings
warnings.filterwarnings("ignore")

In [7]:
# Splitting the data into the chunks

splitter = RecursiveCharacterTextSplitter(separators = ["\n\n","\n"," "], chunk_size = 1000 , chunk_overlap = 50 )
chunks = splitter.split_documents(data)
len(chunks)

88

In [8]:
# Loading model to create the embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Creating and indexed database
chroma_database = Chroma.from_documents(chunks,
                                      embedding_model,
                                      persist_directory = 'chroma_db')

In [10]:
llm = CTransformers(model='TheBloke/Llama-2-7B-Chat-GGML',
                      model_type='llama',
                      config={'max_new_tokens': 600,
                              'temperature': 0.01,
                              'context_length': 2000}
                      )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.ggmlv3.q2_K.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

In [11]:

# Defining a QnA chain
QnA = RetrievalQA.from_chain_type(llm = llm,
                                 chain_type = 'stuff',
                                 retriever = chroma_database.as_retriever(),
                                 verbose = False)

In [12]:
# Defining function to fetch documents according to a query
def get_answers(QnA, query):
    answer = QnA.run(query)
    print(f"\033[1mQuery:\033[0m {query}\n")
    print(f"\033[1mAnswer:\033[0m ", answer)

In [13]:

query = """who forces have regained the initiative in Ukraine ?"""
get_answers(QnA, query)

Query: who forces have regained the initiative in Ukraine ?

Answer:   According to the text, it is the Russian forces that have regained the initiative in Ukraine.
